---
title : "Quick start guide"
execute:
    output: asis
---

Waar begin je met het implementeren van de toolbox continu inzicht? Dat kan best wisselend zijn en is per implementatie verschillend. Dit is namelijk afhankelijk van de wensen die je als gebruiker hebt en de randvoorwaarden van de omgeving waar in je implementeerd. 
Continu Inzicht is een operationeel systeem dus het ophalen de huidige belastingen is een goed begin punt.

## Loads (belastingen)

In toolbox continu inzicht zijn een aantal bronnen voor belastingen opgenomen. Je kan natuurlijk ook data uit andere bronnen ophalen. Voor dit voorbeeld worden waterstanden uit RWS water webservices gebruikt. Alle voorbeelden zijn onder [`Belastingen inladen` te vinden](2.belastingen_inladen.ipynb)) 

In [ ]:
from pathlib import Path
import pandas as pd

path = Path.cwd() / "data_sets" / "103.quick_start_guide"

Met de volgende configuratie:
```yaml
GlobalVariables:
    rootdir: "data_sets/103.quick_start_guide"
    moments: [ -24, 0, 24, 48 ]
    aquo_alias:
        WATHTE [m][NAP][OW]: "WATHTE"

    LoadsWaterwebservicesRWS:
        parameters: [ "WATHTE" ]
        MISSING_VALUE: 999

DataAdapter:
    default_options:
        csv:
            sep: ";"
    belasting_locaties_waterwebservices_RWS:
        type: csv
        path: "locaties.csv"
    waterstanden_waterwebservices:
        type: csv
        path: "hidden_waterstanden_waterwebservices_rws.csv"


```

In [ ]:
from toolbox_continu_inzicht.base.data_adapter import Config, DataAdapter

config = Config(config_path=path / "loads.yaml")
config.lees_config()
data_adapter = DataAdapter(config=config)

In [ ]:
from toolbox_continu_inzicht.loads import LoadsWaterwebservicesRWS

waterwebservices_rws = LoadsWaterwebservicesRWS(data_adapter=data_adapter)
waterwebservices_rws.run(
    input="belasting_locaties_waterwebservices_RWS",
    output="waterstanden_waterwebservices",
)

Deze reek kan gebruikt worden om de waterstanden te tonen aan de gebruiker

In [ ]:
df_waterstanden = data_adapter.input("waterstanden_waterwebservices")

df_waterstanden["date_time"] = pd.to_datetime(df_waterstanden["date_time"])
df_waterstanden.plot(
    x="date_time", y="value", title="Water standen van Waterwebservices"
);

## Loads to moments

De beslastingen worden gebruikt om een kans uit te rekenen. 
Deze berekening kan elk moment in de tijd, deze tijdsstippen leggen we vast en heten: 'moments'. 
De moments worden (uren) gedefineerd in de configuratie. 

```yaml
GlobalVariables:
    rootdir: "data_sets/103.quick_start_guide"
    moments: [ -24, 0, 24, 48 ]

DataAdapter:
    default_options:
        csv:
            sep: ";"
    waterstanden_waterwebservices:
        type: csv
        path: "waterstanden_waterwebservices_rws.csv"
    moments_waterstanden_waterwebservices:
        type: csv
        path: "momenten_waterstanden.csv"

```

In [ ]:
config = Config(config_path=path / "loads_to_moments.yaml")
config.lees_config()
data_adapter = DataAdapter(config=config)

In [ ]:
from toolbox_continu_inzicht.loads import LoadsToMoments

load_moments = LoadsToMoments(data_adapter=data_adapter)
load_moments.run(
    input="waterstanden_waterwebservices",
    output="moments_waterstanden_waterwebservices",
)

In [ ]:
df_moments = data_adapter.input("moments_waterstanden_waterwebservices")

df_moments["date_time"] = pd.to_datetime(df_moments["date_time"])
ax = df_moments.plot(
    x="date_time",
    y="value",
    title="Water standen van Waterwebservices",
    lw=0,
    markersize=5,
    marker="o",
)
df_waterstanden.plot(x="date_time", y="value", ax=ax);

In [ ]:
df_moments[
    ["date_time", "value", "hours", "measurement_location_description", "value_type"]
]